In [ ]:
# Building a Simple UDP Server
import socket

# create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP Server is ready to receive API data...")

while True:
    data, client_address = server_socket.recvfrom(2048)
    print(f"Received data from {client_address}: {data.decode()}")

UDP Server is ready to receive API data...


In [ ]:
#  Building a Simple UDP Client
import socket


client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

server_address = ('localhost', 65433)
message = b"Hello, UDP Server!"

# send message to the server
client_socket.sendto(message, server_address)

client_socket.close()

In [ ]:
# Exercise 1: UDP Chat Application
# Server Code:
import socket


server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP Chat Server is ready to receive messages...")

while True:
    data, client_address = server_socket.recvfrom(2048)
    print(f"Received from {client_address}: {data.decode()}")
    # broadcast the message to all clients 

In [ ]:
# Exercise 1: UDP Chat Application
# Client Code:
import socket


client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

server_address = ('localhost', 65433)

while True:
    message = input("Enter message: ")
    client_socket.sendto(message.encode(), server_address)
    # Receive response 
    data, _ = client_socket.recvfrom(2048)
    print(f"Received from server: {data.decode()}")

In [ ]:
# Exercise 2: Dictionary of User IP Addresses
# Server Code:
import socket

# create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP Server is ready to receive messages...")

users = {}  # dictionary to store user IP addy

while True:
    data, client_address = server_socket.recvfrom(2048)
    users[client_address] = data.decode()  # store user message
    print(f"Users: {users}")

In [ ]:
# Exercise 3: Authentication
# Server Code:
import socket


server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP Server is ready to receive messages...")

# credentials for demonstration
valid_username = "user"
valid_password = "pass"

while True:
    # receive username
    username, client_address = server_socket.recvfrom(2048)
    username = username.decode()

    # receive password
    server_socket.sendto(b"Enter password:", client_address)
    password, _ = server_socket.recvfrom(2048)
    password = password.decode()

    # validate credentials
    if username == valid_username and password == valid_password:
        server_socket.sendto(b"Authentication successful!", client_address)
    else:
        server_socket.sendto(b"Authentication failed!", client_address)

In [ ]:
# Exercise 3: Authentication
# Client Code:
import socket


client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

server_address = ('localhost', 65433)

# send username
username = input("Enter username: ")
client_socket.sendto(username.encode(), server_address)

# receive password prompt
data, _ = client_socket.recvfrom(2048)
print(data.decode())

# send password
password = input("Enter password: ")
client_socket.sendto(password.encode(), server_address)

# receive authentication result
data, _ = client_socket.recvfrom(2048)
print(data.decode())

client_socket.close()

In [ ]:
# Exercise 4: Encrypt Messages
pip install cryptography

In [ ]:
# Exercise 4: Encrypt Messages
# Client Code:
from cryptography.fernet import Fernet
import socket

# generate a key
key = Fernet.generate_key()
cipher = Fernet(key)

# create a UDP socket
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

server_address = ('localhost', 65433)

# encrypt message
message = input("Enter message: ")
encrypted_message = cipher.encrypt(message.encode())

# send encrypted message
client_socket.sendto(encrypted_message, server_address)

client_socket.close()

In [ ]:
# Exercise 4: Encrypt Messages
# Server Code:
from cryptography.fernet import Fernet
import socket

# use the same key as the client
key = b"..."  # replace with the key generated by the client
cipher = Fernet(key)

# create a UDP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))

print("UDP Server is ready to receive encrypted messages...")

while True:
    data, client_address = server_socket.recvfrom(2048)
    decrypted_message = cipher.decrypt(data)
    print(f"Received from {client_address}: {decrypted_message.decode()}")

In [ ]:
# Exercise 5: Compare Temperatures
import socket
import requests

# fetch weather data for Goldsmiths University
api_url_goldsmiths = "https://api.open-meteo.com/v1/forecast?latitude=51.47&longitude=0.0363&current_weather=true"
response_goldsmiths = requests.get(api_url_goldsmiths)

# fetch weather data for the British Library
api_url_british_library = "https://api.open-meteo.com/v1/forecast?latitude=51.53&longitude=0.1276&current_weather=true"
response_british_library = requests.get(api_url_british_library)

if response_goldsmiths.status_code == 200 and response_british_library.status_code == 200:
    temp_goldsmiths = response_goldsmiths.json()["current_weather"]["temperature"]
    temp_british_library = response_british_library.json()["current_weather"]["temperature"]
    message = f"Temperature at Goldsmiths: {temp_goldsmiths}°C\nTemperature at British Library: {temp_british_library}°C"
else:
    message = "Failed to fetch weather data"

# send the weather data using UDP
client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)

client_socket.sendto(message.encode(), server_address)
print("Weather data sent!")

client_socket.close()
